In [5]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [6]:
from utils.imports import *
from baseline_direct_subtraction import *

# get legend
from dataloader_17 import py_labels as labels
a_labels = list(labels.values())
a_labels.insert(0, 'Unknown')
a_labels = np.array(a_labels)
# label_patches = get_legend(a_labels)

PATH = Path('data/cm')
path_x_512_1 = PATH/'2017'
path_x_512_2 = PATH/'2018'
path_y_512 = PATH/'labels'
path_test_x_512_1 = path_x_512_1
path_test_x_512_2 = path_x_512_2
path_stats = PATH/'stats.npy'

sz = 512
test_size = 0.2
gpu_start = 7
world_size = 1
bs = world_size * 20
num_workers = 0

In [ ]:
%run train_cm {str(path_x_512_1)} {str(path_x_512_2)} {str(path_y_512)} {str(PATH)} {str(path_stats)}\
    {bs} {sz} {gpu_start} --world_size={world_size} --path_test_x_1={path_test_x_512_1} --path_test_x_2={path_test_x_512_2}\
    --path_test_y={path_y_512} --num_workers={num_workers}

--Return--
None
> /data/ywx/home/landcover-diff/dataloader_cm.py(12)__init__()
     10         self.transform_no.tfms = [o for o in transform.tfms]
     11         for o in self.transform_no.tfms: o.tfm_y = TfmType.NO
---> 12         set_trace()
     13 
     14     def get_x(self, i):



ipdb>  [o.tfm_y for o in self.transform_class.tfms]


[<TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>]


ipdb>  [o.tfm_y for o in self.transform_no.tfms]


[<TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>]


## Test data

In [5]:
from baseline_direct_subtraction import *
names = [o.split('/')[-1][5:-4] for o in md17.test_ds.fn_x]
save_path_17 = PATH/'baseline_direct_subtraction/17_8ch_final'; save_path_17.mkdir(exist_ok=True)
save_path_18 = PATH/'baseline_direct_subtraction/18_8ch_final'; save_path_18.mkdir(exist_ok=True)

In [13]:
# %%time
# save_preds(learner, md17.test_dl, save_path_17, names, num_workers=40)

waiting for writing to disk
CPU times: user 32min 58s, sys: 2min 36s, total: 35min 34s
Wall time: 1min 23s


In [14]:
# %%time
# save_preds(learner, md18.test_dl, save_path_18, names, num_workers=80)

waiting for writing to disk
CPU times: user 34min 3s, sys: 2min 37s, total: 36min 41s
Wall time: 1min 13s


In [6]:
# %%time
fns_p1, fns_p2 = np.array(sorted(save_path_17.glob('*.npz'))), np.array(sorted(save_path_18.glob('*.npz')))
# overlap_rank = rank_overlap(fns_p1, fns_p2, max_workers=16)
# np.save(str(PATH/'overlap_rank'), overlap_rank)
overlap_rank = np.load(str(PATH/'overlap_rank.npy'))
fns_p_ranked_1, fns_p_ranked_2 = fns_p1[overlap_rank], fns_p2[overlap_rank]
fns_ims_ranked_1, fns_ims_ranked_2 = np.array(md17.test_ds.fn_x)[overlap_rank], np.array(md18.test_ds.fn_x)[overlap_rank]

## Everything related to buildings

In [7]:
from baseline_direct_subtraction import *

In [21]:
%%time
mask_fn_buildings_related = filter_related_to_class(fns_p_ranked_1, fns_p_ranked_2, 3, threshold=0.65)

CPU times: user 13.3 s, sys: 29.4 s, total: 42.8 s
Wall time: 1min 9s


In [26]:
fns_ims_buildings1, fns_ims_buildings2 = fns_ims_ranked_1[mask_fn_buildings_related], fns_ims_ranked_2[mask_fn_buildings_related]
fns_p_buildings1, fns_p_buildings2 = fns_p_ranked_1[mask_fn_buildings_related], fns_p_ranked_2[mask_fn_buildings_related]

In [ ]:
N = 20
for i in range(0, len(fns_ims_buildings1), N):
# for i in range(0, N, N):
    old, new, blended, classes_with_biggest_area = most_prominent_difference_fns(fns_ims_buildings1[i:i+N], fns_p_buildings1[i:i+N],
                                                                                 fns_ims_buildings2[i:i+N], fns_p_buildings2[i:i+N],
                                                                                 alpha=0.7, threshold=0.6)
    (rgb_old, color_old), (rgb_new, color_new), (blend_old, blend_new) = old, new, blended
    
    additional_titles = [f'{a_labels[o1]}=>{a_labels[o2]}' for (o1, o2) in zip(*classes_with_biggest_area)]
    plot_blended_with_rgb(rgb_old, rgb_new, blend_old, additional_titles=additional_titles, legend=a_labels)

In [59]:
# transformations
tfm_y, tfm_y_n = TfmType.CLASS, TfmType.NO
aug_tfms = [o for o in transforms_top_down]
aug_tfms.append(Copy())
for t in aug_tfms: t.tfm_y = tfm_y
tfms_c = tfms_from_stats(None, sz, aug_tfms=aug_tfms, tfm_y=tfm_y, norm_y=False)
tfms_n = tfms_from_stats(None, sz, aug_tfms=aug_tfms, tfm_y=tfm_y_n, norm_y=False)
for o in tfms_c: o.tfms.append(Copy()) # fix pytorch negative error
for o in tfms_n: o.tfms.append(Copy()) # fix pytorch negative error

In [16]:
[o.tfm_y for o in transforms_top_down]

[<TfmType.NO: 1>, <TfmType.NO: 1>, <TfmType.NO: 1>]

In [8]:
aug_tfms = [o for o in transforms_top_down]
tfms_c = tfms_from_stats(None, sz, aug_tfms=aug_tfms, tfm_y=TfmType.CLASS, norm_y=False)

In [20]:
t = tfms_c[0]
[o.tfm_y for o in t.tfms]

[<TfmType.CLASS: 4>,
 <TfmType.NO: 1>,
 <TfmType.NO: 1>,
 <TfmType.NO: 1>,
 <TfmType.CLASS: 4>,
 <TfmType.CLASS: 4>]

In [78]:
t = tfms_c[]
for o in t.tfms: print(o, o.tfm_y)

<fastai.transforms.Scale object at 0x7f896b910128> TfmType.NO
<fastai.transforms.RandomRotate object at 0x7f896bf78b70> TfmType.CLASS
<fastai.transforms.RandomLighting object at 0x7f896bf78be0> TfmType.CLASS
<fastai.transforms.RandomDihedral object at 0x7f896bf78c88> TfmType.CLASS
<utils.misc.Copy object at 0x7f896bc50588> TfmType.CLASS
<fastai.transforms.RandomCrop object at 0x7f896b910ba8> TfmType.NO
<fastai.transforms.ChannelOrder object at 0x7f896b9104a8> TfmType.NO
